---
#1. Preparation
##1.1 Download Stable-Diffusion Checkpoint

In [1]:
%%shell
mkdir -p pretrained/
cd pretrained/
wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt
cd ..

--2023-07-31 13:14:59--  https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt
Resolving huggingface.co (huggingface.co)... 13.35.7.5, 13.35.7.81, 13.35.7.38, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.5|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/4c/37/4c372b4ebb57bbd02e68413d4951aa326d4b3cfb6e62db989e529c6d4b26fb21/14749efc0ae8ef0329391ad4436feb781b402f4fece4883c7ad8d10556d8a36a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd-v1-4-full-ema.ckpt%3B+filename%3D%22sd-v1-4-full-ema.ckpt%22%3B&Expires=1691068499&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MTA2ODQ5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Yy8zNy80YzM3MmI0ZWJiNTdiYmQwMmU2ODQxM2Q0OTUxYWEzMjZkNGIzY2ZiNmU2MmRiOTg5ZTUyOWM2ZDRiMjZmYjIxLzE0NzQ5ZWZjMGFlOGVmMDMyOTM5MWFkNDQzNmZlYjc4MWI0MDJmNGZlY2U0ODgzYzdhZD

##1.2 Clone CelebBasis Code from [GitHub](https://github.com/ygtxr1997/CelebBasis/tree/main)

In [2]:
!git clone https://github.com/ygtxr1997/CelebBasis.git

Cloning into 'CelebBasis'...
remote: Enumerating objects: 788, done.
remote: Counting objects: 100% (788/788), done.
remote: Compressing objects: 100% (635/635), done.
remote: Total 788 (delta 127), reused 739 (delta 97), pack-reused 0
Receiving objects: 100% (788/788), 16.82 MiB | 9.15 MiB/s, done.
Resolving deltas: 100% (127/127), done.


## 1.3 Download the StyleGAN Datasets and Third-Party Pretrained Weights from HuggingFace

In [3]:
# dataset
!mkdir -p ./datasets/ && cd ./datasets/ && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/stylegan.tar
!cd ./datasets/ && tar -xf stylegan.tar && rm stylegan.tar
# face alignment model
!mkdir -p ./pretrained/PIPNet/ && mkdir -p ./pretrained/glint360k_cosface_r100_fp16_0.1/
!cd ./pretrained/PIPNet/ && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/PIPNet/FaceBoxesV2.pth && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/PIPNet/epoch59.pth
!cp ./pretrained/PIPNet/* ./CelebBasis/evaluation/face_align/PIPNet/weights
# face recognition model
!cd ./pretrained/glint360k_cosface_r100_fp16_0.1/ && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/glint360k_cosface_r100_fp16_0.1/backbone.pth
!mkdir -p ./CelebBasis/weights/
!cp -r ./pretrained/glint360k_cosface_r100_fp16_0.1/ ./CelebBasis/weights/

##1.4 *(Optional) Copy StyleGAN Datasets and Third-Party Pretrained Weights from Your Own Google Drive*
*If you have any downloading problem in (Step 1.3), you can copy the datasets and pretrained models to your own Google Drive as follows.*

You should first copy [CelebBasis Google Drive](https://drive.google.com/drive/folders/181pJ3Y2Ffk0R8jVcErPT611aWQLCz7D9?usp=sharing) to your own [Google Drive](https://drive.google.com/drive/my-drive).
The Google Drive directory tree will be like:
```shell
My Drive/
 |--CelebBasis/
  |--datasets/
  |--pretrained/
```
Then, the following cell mounts your own Google Drive to this Colab Notebook and copies the datasets and weights files from the mounted folder.

In [4]:
# @title Use Googgle Drive? { display-mode: "form" }

use_google_drive = False # @param ["False", "True"]
print('Use Google Drive:', use_google_drive)
if use_google_drive:
  import os
  from google.colab import drive
  drive.mount('/content/drive')
  os.system('cp -r /content/drive/MyDrive/CelebBasis/datasets/ ./')
  os.system('cp -r /content/drive/MyDrive/CelebBasis/pretrained/* ./pretrained/')
  drive.flush_and_unmount()
  os.system('cd ./datasets && tar -xf stylegan.tar && rm stylegan.tar')
  os.system('cp ./pretrained/PIPNet/* ./CelebBasis/evaluation/face_align/PIPNet/weights')
  os.system('mkdir -p ./CelebBasis/weights/')
  os.system('cp -r ./pretrained/glint360k_cosface_r100_fp16_0.1/ ./CelebBasis/weights/')
else:
  pass

Use Google Drive: False


## 1.5 Use PIP to Install the Python Dependencies and Uninstall the Needless Modules (which are preinstalled by Google Colab but not used)

In [20]:
!pip install blinker==1.6.2 --ignore-installed
!pip uninstall -r ./CelebBasis/requirements_uninstall.txt -y
!mkdir -p pip_installed/ && cd pip_installed/ && pip install -r ../CelebBasis/requirements.txt
!pip list |grep torch

  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
Obtaining clip from git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33#egg=clip (from -r ../CelebBasis/requirements.txt (line 18))
  Skipping because already up-to-date.
  Preparing metadata (setup.py) ... done
Obtaining taming_transformers from git+https://github.com/CompVis/taming-transformers.git@3ba01b241669f5ade541ce990f7650a3b8f65318#egg=taming_transformers (from -r ../CelebBasis/requirements.txt (line 141))
  Skipping because already up-to-date.
  Preparing metadata (setup.py) ... done
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached aiohttp-3.8.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached albumentations-1.1.0-py3-none-any.whl (102 kB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached asttokens-2.2.1-py2.py3-none-any.whl (26 kB)
  Using cached attrs-22.1.0-py2.py3-none-any.whl (58 kB)
  Using cached autofaiss-2.15.6-py

---
#2. Crop and Align Faces

In [31]:
!cd CelebBasis && chmod +x *.sh && bash ./00_align_face.sh ../datasets/stylegan/stylegan3-r-ffhq-1024x1024 ../datasets/stylegan/ffhq

/content/CelebBasis/evaluation/face_align/PIPNet/FaceBoxesV2/utils/build.py:13: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.core import setup
running build_ext
skipping 'nms/cpu_nms.c' Cython extension (up-to-date)
/content/CelebBasis
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
alignment model loaded


#3. Train
* Limited by Google Colab V100 GPU Memory ($<$16GB), we have to modify the `configs/stable-diffusion/aigc_id_colab.yaml` as follows:
```yaml
model:
  params:
    personalization_config:
      params:
        momentum: 0.9  # default: 0.99
data:
  params:
    batch_size: 1  # default: 2
    num_workers: 2  # default: 8
lightning:
  modelcheckpoint:
    params:
      every_n_train_steps: 400  # default: 200
  callbacks:
    image_logger:
      params:
        batch_frequency: 1200  # default: 600
  trainer:
    max_steps: 800  # default: 1600
```
* These modifications may increase the training time and downgrade the performance slightly.

In [22]:
!cd CelebBasis && bash ./01_start_train.sh ../pretrained/sd-v1-4-full-ema.ckpt ./configs/stable-diffusion/aigc_id_colab.yaml

Global seed set to 23
Running on GPUs 0,
Loading model from ../pretrained/sd-v1-4-full-ema.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Downloading: "https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty_with_aug/checkpoint_liberty_with_aug.pth" to /root/.cache/torch/hub/checkpoints/checkpoint_liberty_with_aug.pth
100% 5.10M/5.10M [00:00<00:00, 63.0MB/s]
Downloading: 100% 939k/939k [00:00<00:00, 5.43MB/s]
Downloading: 100% 512k/512k [00:00<00:00, 1.02MB/s]
Downloading: 100% 389/389 [00:00<00:00, 293kB/s]
Downloading: 100% 905/905 [00:00<00:00, 660kB/s]
Downloading: 100% 4.41k/4.41k [00:00<00:00, 3.47MB/s]
Downloading: 100% 1.59G/1.59G [00:23<00:00, 71.4MB/s]
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializin

#4. Test
Modify the following `"trainingYYYY-MM-DDTHH-MM-SS_celebbasis/"` to the project folder under `./CelebBasis/logs/` and run the test.

In [29]:
# @title Choose a project folder from the following list:  { display-mode: "form" }

print('=' * 10, ' Choose a project folder from the following list ', '=' * 10)
!ls -hlrt ./CelebBasis/logs/

==========  Choose a project folder from the following list  ==========
total 4.0K
drwxr-xr-x 6 root root 4.0K Jul 31 13:50 training2023-07-31T13-40-17_celebbasis


In [30]:
# @title Choose your project folder to be tested: { display-mode: "form" }

project_folder = "trainingYYYY-MM-DDTHH-MM-SS_celebbasis/" # @param
prompt_file = "./infer_images/example_prompt.txt"  # @param
batch_size = 4 # @param

print('=' * 10, ' testing ', '=' * 10)
!cd CelebBasis && bash ./02_start_test.sh "../pretrained/sd-v1-4-full-ema.ckpt" $prompt_file $project_folder $batch_size

==========  testing  ==========
logs/training2023-07-31T13-40-17_celebbasis/configs/training2023-07-31T13-40-17-project.yaml
embeddings_gs-799.pt
Global seed set to 42
Loading model from ../pretrained/sd-v1-4-full-ema.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.20.self_attn.v_proj.bias', 'vision_model.encoder.layers.1.mlp.fc1.bias', 'vision_model.encoder.layers.21.self_attn.k_proj.bias', 'vision_model.encoder.layers.4.self_attn.v_proj.bias', 'vision_model.encoder.layers.12.layer_norm2.bias', 'vision_model.encoder.layers.8.self_attn.q_proj.weight', 'vision_model.encoder.layers.21.layer_norm2.weight', 'vision_

The generated images will be output under `./CelebBasis/outputs/trainingYYYY-MM-DDTHH-MM-SS_celebbasis/`. Enjoy it!

In [9]:
!nvidia-smi

Mon Jul 31 13:25:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------